In [9]:
# Custom tool : 에이전트에 새로운 능력(@tool) 추가하기

from langchain.agents import initialize_agent, AgentType
# from langchain.llms import OpenAI
from langchain.agents.tools import tool

In [15]:
# 새로운 능력 : @tool - 실시간 주가 검색하기
from pydantic import BaseModel
import yfinance as yf

class StockRequest(BaseModel):
    symbol: str  # 종목 코드

# 함수 정의 및 데코레이션 적용
@tool
def get_korean_stock_price(input: StockRequest) -> str:
    """
    yfinance를 사용하여 한국 주식의 가격을 조회합니다.
    최대한 안정적이고 빠른 방법을 사용합니다.
    """
    code = input.symbol.strip()

    # 접미사 자동 판별
    if code.endswith(".KQ") or code.endswith(".KS"):
        ticker = code
    else:
        if code.startswith(("0", "1", "2")):
            ticker = code + ".KQ"
        else:
            ticker = code + ".KS"

    try:
        stock = yf.Ticker(ticker)

        # 우선 fast_info를 먼저 사용
        price = stock.fast_info.get("last_price") or stock.fast_info.get("regular_market_price")
        name = stock.fast_info.get("shortName") or ticker

        # fast_info 실패 시 history 사용 (최근 1분봉 데이터)
        if price is None:
            df = stock.history(period="1d", interval="1m")
            if not df.empty:
                price = df["Close"].iloc[-1]
            else:
                raise ValueError("가격 정보를 가져올 수 없습니다.")

        return f"{name}({ticker})의 현재 가격은 {price:.2f}원입니다."

    except Exception as e:
        return f"오류: {e}"

ModuleNotFoundError: No module named 'yfinance'

In [12]:
# 에이전트 초기화 : 언어모델 / 도구 목록 / 에이전트 초기화 + 에이전트 실행

# 언어모델 
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# pip install langchain-openai


llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    # model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    model = "lmstudio-community/gemma-2-2b-it-GGUF",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # 스트림 출력 콜백
)

# 도구 목록
tools = []
tools.append(get_korean_stock_price)   # 메서드 이름

# 에이전트 초기화
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 에이전트 실행
result = agent.run("Samsung electronics stock price?")
print(result)



> Entering new AgentExecutor chain...
Thought: I need to get the stock price of Samsung Electronics. 
Action: get_korean_stock_price
Action Input: __main__.StockRequest(symbol='005930') Thought: I need to get the stock price of Samsung Electronics. 
Action: get_korean_stock_price
Action Input: __main__.StockRequest(symbol='005930') 

ValidationError: 1 validation error for get_korean_stock_price
input
  Input should be a valid dictionary or instance of StockRequest [type=model_type, input_value="__main__.StockRequest(symbol='005930')", input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type